Hi, everyone. My name is Sharon, an aspiring data scientist and Machine learning engineer. I got accepted to kaggle X fellowship, where we were given a project, to work on building a conversational chatbot using Gemma. As someone new to NLP and LLM, it took a while before choosing what type of chatbot to build. Eventually, i decided to build a conversational(questiona and answer) chatbot that help data enthusiast in their data journey, whether they are new to the space or an expert in the jorney. 

Kindly note that its not perfect, but i hope to rework it and make it more refined. Please, feel free to drop any comment or corrections. Be on the lookout for the deployed link. Also, i want to thank my mentor, @Osama Sarhan , for his guidances through out the project development.


Installing libraries

In [1]:
!pip install sentence-transformers
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.6 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 10.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 32.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install langchain langchain-community langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following 

Importing and creating a backend

In [3]:
import os

os.environ["KERAS_BACKEND"] = "torch" # "jax"  or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [4]:
import keras
import keras_nlp

Loading the dataset

In [5]:
import json

data = []
with open('/kaggle/input/ml-dataset/ML_dataset (1).json') as file:
    feature_list = json.load(file)
    # Filter out examples with context, to keep it simple.
    for features in feature_list:
        template = "Instruction:\n{line}"
        data.append(template.format(**features))

Loading the gemma model

In [6]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

Testing the model

In [7]:
instruction = "What is data science?"
prompt = template.format(line=instruction, response="")

print(gemma_lm.generate(prompt, max_length=100))

Instruction:
What is data science?
Data science is the process of collecting, cleaning, and analyzing data to extract useful information. Data science is a combination of statistics, mathematics, and computer science.
Data science is a process of collecting, cleaning, and analyzing data to extract useful information. Data science is a combination of statistics, mathematics, and computer science.
Data science is a process of collecting, cleaning, and analyzing data to extract useful information. Data science is a combination of


In [8]:
prompt = template.format(
    line="What is data engineering?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=150))

Instruction:
What is data engineering?
Data engineering is the process of collecting, storing, and analyzing data to extract insights and make informed decisions. It involves the design, development, and maintenance of data systems, as well as the creation of data-driven applications and services. Data engineering is a rapidly growing field, as organizations increasingly rely on data to drive decision-making and improve business performance.
Data engineering is a complex and multifaceted discipline, requiring a deep understanding of data structures, algorithms, and software engineering principles. It involves a wide range of skills, including data modeling, data warehousing, data mining, and data visualization. Data engineers work closely with data scientists, software developers, and business analysts to design and implement data-driven solutions


Fine tuning and fitting with Lora

In [9]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [10]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

1554/1554 ━━━━━━━━━━━━━━━━━━━━ 1385s 890ms/step - loss: 0.1758 - sparse_categorical_accuracy: 0.4269


In [11]:
template = "Instruction:\n{line}\n\nResponse:\n{response}"
prompt = template.format(
    line="What is data science?",
    response="",
)
print(gemma_lm.generate(prompt, max_length= 50))

Instruction:
What is data science?

Response:
Data science is the process of collecting, analyzing, and interpreting data to gain insights and make informed decisions. It involves using statistical methods, machine learning algorithms, and data visualization techniques to extract


In [12]:
prompt = template.format(
    line="What is data engineering?",
    response="",
)
print(gemma_lm.generate(prompt, max_length= 50))

Instruction:
What is data engineering?

Response:
Data engineering is the process of collecting, cleaning, and analyzing data to extract insights and make data-driven decisions.


**Implementing RAG Technique**

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from IPython.display import display, Markdown
import json
from langchain.schema import Document


In [14]:
class RAGSystem:
    """Retrieval-Augmented Generation System with Keras Model."""
    def __init__(self, gemma_lm, num_retrieved_docs=2, data_path="/kaggle/input/ml-dataset/ML_dataset (1).json"):
        # Set up the model and retriever parameters
        self.num_docs = num_retrieved_docs
        self.gemma_lm = gemma_lm
        self.template = "\n\nQuestion:\n{question}\n\nPrompt:\n{prompt}\n\nAnswer:\n{answer}\n\nContext:\n{context}"

         # Load JSON data without jq
        with open(data_path, 'r') as file:
            raw_data  = json.load(file)
            
        # Format JSON data to a list of Document objects with `page_content`
        documents = [Document(page_content=doc.get("line", ""), metadata={}) for doc in raw_data]
        
        # Text splitting
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
        all_splits = text_splitter.split_documents(documents)

        # Embeddings for retrieval
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
        self.vector_db = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")
        self.retriever = self.vector_db.as_retriever()

    def retrieve(self, query):
        """Retrieve top k similar documents based on the query."""
        docs = self.retriever.invoke(query)
        return docs
    
    
    def query(self, query):
        #"""Generate an answer based on retrieved documents and query."""
        # Retrieve context documents
        context_docs = self.retrieve(query)
        context = "".join([doc.page_content for doc in context_docs[:self.num_docs]])
       
        # Generate response with the model
        prompt = f"{query}\n\nContext:\n{context}"
        generated_output = self.gemma_lm.generate(prompt, max_length=150)

        # Answer generation
        #generated_output = self.gemma_lm.generate(prompt,max_length= 250)
        return self.template.format(question=query, 
                                    prompt=prompt, 
                                    answer=generated_output,
                                   context=context)

In [15]:
def colorize_text(text):
    for word, color in zip(["Question", "Prompt", "Answer", "Context"], ["blue", "magenta", "green", "Purple"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [16]:
rag_system = RAGSystem(gemma_lm)

/tmp/ipykernel_30/2033221000.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Testing the system

In [17]:
# testing the query
question = "What is data science?"
response = rag_system.query(question)
display(Markdown(colorize_text(response)))



**<font color='blue'>Question:</font>**
What is data science?

**<font color='magenta'>Prompt:</font>**
What is data science?

**<font color='Purple'>Context:</font>**
What is data science?Course - What is data science?

**<font color='green'>Answer:</font>**
What is data science?

**<font color='Purple'>Context:</font>**
What is data science?Course - What is data science?

Data science is a field that combines data analysis, statistics, and machine learning to extract insights from data. It involves collecting, cleaning, and analyzing data to answer specific questions or solve problems. Data scientists use various techniques, such as data mining, statistical analysis, and machine learning algorithms, to extract insights from data.

Data science is a rapidly growing field, with a wide range of applications in various industries, including healthcare, finance, and marketing. It is also used in fields such as artificial intelligence, robotics, and computer vision.

Data science is a highly technical field, and it requires a deep understanding of mathematics, statistics,

**<font color='Purple'>Context:</font>**
What is data science?Course - What is data science?

In [18]:
# trying another question
question = "What is Machine learning?"
response = rag_system.query(question)
display(Markdown(colorize_text(response)))



**<font color='blue'>Question:</font>**
What is Machine learning?

**<font color='magenta'>Prompt:</font>**
What is Machine learning?

**<font color='Purple'>Context:</font>**
What is Machine Learning?Course - What is machine learning?

**<font color='green'>Answer:</font>**
What is Machine learning?

**<font color='Purple'>Context:</font>**
What is Machine Learning?Course - What is machine learning?

Machine learning is a subset of artificial intelligence (AI) that focuses on developing algorithms that can learn from data and make predictions without being explicitly programmed.

Machine learning algorithms are used in a wide range of applications, including image and speech recognition, natural language processing, and data mining.

Machine learning is a rapidly evolving field, and new algorithms and techniques are being developed all the time.

Machine learning is a subset of artificial intelligence (AI) that focuses on developing algorithms that can learn from data and make predictions without being explicitly programmed.

Machine learning algorithms are used in a wide range of applications, including image and speech recognition, natural

**<font color='Purple'>Context:</font>**
What is Machine Learning?Course - What is machine learning?

In [19]:
question = "recommend projects on data science?"
response = rag_system.query(question)
display(Markdown(colorize_text(response)))



**<font color='blue'>Question:</font>**
recommend projects on data science?

**<font color='magenta'>Prompt:</font>**
recommend projects on data science?

**<font color='Purple'>Context:</font>**
Here are some popular websites where you can find project ideas, datasets, and guidance to develop your data science and machine learning skills:Course - Recommend a good data Science Road map?

**<font color='green'>Answer:</font>**
recommend projects on data science?

**<font color='Purple'>Context:</font>**
Here are some popular websites where you can find project ideas, datasets, and guidance to develop your data science and machine learning skills:Course - Recommend a good data Science Road map?

* Kaggle - Kaggle is a popular platform for data science competitions and projects. It provides a wide range of datasets and challenges for users to explore and learn from.
* DataCamp - DataCamp is a platform that offers online courses and interactive learning materials for data science and machine learning. It provides a variety of projects and challenges for users to work on.
* Data Science Central - Data Science Central is a community-driven website that provides a variety of resources and tools for data science and machine learning

**<font color='Purple'>Context:</font>**
Here are some popular websites where you can find project ideas, datasets, and guidance to develop your data science and machine learning skills:Course - Recommend a good data Science Road map?

In [20]:
question = "what are the skills of a data scientist?"
response = rag_system.query(question)
display(Markdown(colorize_text(response)))



**<font color='blue'>Question:</font>**
what are the skills of a data scientist?

**<font color='magenta'>Prompt:</font>**
what are the skills of a data scientist?

**<font color='Purple'>Context:</font>**
In short, a data scientist must be able to:What does a data scientist do?

**<font color='green'>Answer:</font>**
what are the skills of a data scientist?

**<font color='Purple'>Context:</font>**
In short, a data scientist must be able to:What does a data scientist do?

* Data collection and data preparation
* Data analysis
* Data visualization
* Data modeling
* Data storytelling
* Data ethics

What is data science?

* Data science is the process of collecting, analyzing, and interpreting data to extract insights and make predictions.
* Data science is a combination of skills, including statistics, programming, and data visualization.
* Data science is a rapidly growing field, with many job opportunities in both the private and public sectors.

What is data science?

* Data science is the process of collecting, analyzing, and interpreting data to

**<font color='Purple'>Context:</font>**
In short, a data scientist must be able to:What does a data scientist do?

In [21]:
question = "What is data engineering?"
response = rag_system.query(question)
display(Markdown(colorize_text(response)))



**<font color='blue'>Question:</font>**
What is data engineering?

**<font color='magenta'>Prompt:</font>**
What is data engineering?

**<font color='Purple'>Context:</font>**
What is data science?What is the difference between data science and data engineering?

**<font color='green'>Answer:</font>**
What is data engineering?

**<font color='Purple'>Context:</font>**
What is data science?What is the difference between data science and data engineering?What is data engineering?Data engineering is the process of building and maintaining data pipelines that move data from its source to its destination. Data engineers are responsible for designing, building, and maintaining data pipelines that move data from its source to its destination.

Data engineering is a critical role in the data science process. Data engineers are responsible for designing, building, and maintaining data pipelines that move data from its source to its destination. Data engineers are responsible for designing, building, and maintaining data pipelines that move data from its source to its destination.

Data engineering is a critical role in the data science process. Data engineers are

**<font color='Purple'>Context:</font>**
What is data science?What is the difference between data science and data engineering?